# Optimizing Entanglement Distillation

Below, we provide a number of examples illustrating the use of the Julia module "EntanglementDist" which implements the methods from arXiv:... For further information, please refer to the full documentation of this Julia module.

It is assumed you have already installed EntanglementDist, as well as the modules required by it. Let's load them.

In [1]:
using Convex;
using SCS;
using EntanglementDist;
importall EntanglementDist;

----
## Example PPT Relaxation

To start, let us compute an upper bound on the fidelity that is achievable by any distillation scheme using realistic operations, when we fix the success probability to be delta. Our first example will use the PPT relaxation method.

The code below consideres three examples of interesting states implemented in EntanglementDist. The first is an EPR pair 

$ 
\qquad \rho = |\Phi\rangle\langle \Phi| \qquad |\Phi\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle_A |0\rangle_B + |1\rangle_A |1\rangle_B\right) 
$

Evidently, when trying to distill the EPR pair to an EPR pair, the only sensible upper bound on the fidelity is 1, since doing nothing produces one EPR pair.

A common example considered in the mathematical quantum information literature is the Werrner state exhibiting many symmetries.

$
\qquad \rho = p |\Phi\rangle\langle \Phi| + (1-p) \frac{I}{4}
$

It is also interesting to consider a state with an othogonal noise term. We will call this the r-State

$
\qquad \rho = p |\Phi\rangle\langle \Phi| + (1-p) |01\rangle \langle 01|
$

### Single copy bounds

Let us now first compute an upper bound when distilling one EPR pair, using exactly one copy of the state.

In [2]:
# Example states

# The werner state with probability p = 0.9
# rho = wernerState(0.9);

# The EPR pair
# rho = maxEnt(2);

# The r State with probability p = 0.9
rho = rState(0.9);

# Dimensions of what is A and B for this state
nA = nB = 2;

# (Local) dimension of the maximally entangled state we want to distill
k = 2;

# Desired probability of success
delta = 0.8;

# Upper bound F on the fidelity
(problem, F, psucc) = pptRelax(rho, nA, nB, k, delta);

print("Original fidelity to the EPR pair ", round(entFidelity(rho),3),"\n");
print("The fidelity for success probability ", delta, " can never exceed ", round(F,3), "\n");

(size(coeff),size(var)) = ((4,4),(4,4))
(size(coeff),size(var)) = ((4,4),(4,4))
(size(coeff),size(var)) = ((4,4),(4,4))
(size(coeff),size(var)) = ((4,4),(4,4))
(size(coeff),size(var)) = ((4,4),(4,4))
(size(coeff),size(var)) = ((4,4),(4,4))
Original fidelity to the EPR pair 0.9
The fidelity for success probability 0.8 can never exceed 0.917


### Multiple copy bounds

To compare let us now consider a situation in which we are given multiple copies $n$ of the state. Clearly, we should only obtain a higher bound.

In [3]:
# Number of copies
n = 2;

# Compute fidelity bound
(problem, F, psucc) = pptRelaxCopies(rho,n, nA, nB, k, delta);


print("Original fidelity to the max Entangled state ", round(entFidelity(copies(rho,n)),3),"\n");
print("For ", n, " copies, the fidelity for success probability ", delta, " can never exceed ", round(F,3), "\n");

(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
Original fidelity to the max Entangled state 0.205
For 2 copies, the fidelity for success probability 0.8 can never exceed 0.999


In [4]:
v = (eVec(4,1) + im * eVec(4,4))/sqrt(2)
rho1= v*v';
p = 0.9;
rho = p * rho1 + (1-p) * eye(4)/4;
isQuantumState(rho)

# rho=wernerState(0.9)
(prob, F2,p2) = pptRelax(rho,2,2,2,0.9);

(size(coeff),size(var)) = ((8,8),(8,8))
(size(coeff),size(var)) = ((8,8),(8,8))
(size(coeff),size(var)) = ((8,8),(8,8))
(size(coeff),size(var)) = ((8,8),(8,8))
(size(coeff),size(var)) = ((8,8),(8,8))


In [5]:
rho = wernerState(0.8)
M= ComplexVariable(4,4);
tv = Variable(1);
problem = maximize(tv);
problem.constraints += tv == trace(M*rho);
problem.constraints += trace(M) == 1;
problem.constraints += M in :SDP;
solve!(problem,check_vexity=false)

(size(coeff),size(var)) = ((8,8),(8,8))


In [6]:
F2

0.9752367327736398 - 1.5134103021103985e-11im

In [7]:
X = [0 1; 1 0]
Y = [0 -im; im 0]
Z = [1 0; 0 -1]
rho = ComplexVariable(2, 2)
prob = maximize(real(trace(rho * X)))
prob.constraints += [rho in :SDP]
prob.constraints += [trace(rho) == 1]
prob.constraints += [trace(rho * Y) == 1/sqrt(2)]
prob.constraints += [trace(rho * Z) == 0.5]
solve!(prob, SCSSolver(verbose=0))

(size(coeff),size(var)) = 

### Producing plots

Let us now illustrate how to produce plots upper bounding the fidelity. We will compare these plots to two well known entanglement distillation schemes in case we have 2 2x2 states to distill, namely the DEJMPS and BBPSSW protocols.

First, let's produce the necessary data. This will take some time to compute.

In [ ]:
# State to distill
rho = wernerState(0.9);
nA = nB = 2;

# Number of copies
n = 2;

# Dimension of max entangled state to produce
k = 2;

# Number of points of success probability
steps = 10;

# Compute all data, this will take a while
pVec = Float64[];
FVec = Float64[];

for j = 0:steps
    p = j * 1/steps; 
    (problem, F, pR) = pptRelaxCopies(rho,n, nA, nB, k, p);
    push!(FVec, F);
    push!(pVec, p);
end

# If the success probability is 0, we can have F=1
FVec[1]=1;

(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,16),(16,16))
(size(coeff),size(var)) = ((16,1

In [ ]:
# Load the graphics package
using Gadfly;

Let's now plot the data. Note that it may take a while for the plot to appear, depending on your computer.

In [ ]:
# Set up the plot
titleName = "Example";

PPTLayer = layer(x=pVec, y=FVec, Geom.line, Theme(default_color=colorant"blue"));

layers = typeof(PPTLayer)[];
names = AbstractString[];
colors = AbstractString[];
push!(layers, PPTLayer)
push!(names, "PPT")
push!(colors, "blue")

# Include schemes in plots if we distill 2 2x2 states
if (n==2) && (nA == 2) && (nB==2)
    (F, p_succ) = deutschParam(rho);
    deutschLayer = layer(y = [F], x = [p_succ], Geom.point, Theme(default_color=colorant"cyan"));
    (F, p_succ) = bennettParam(rho);
    bennettLayer = layer(y = [F], x = [p_succ], Geom.point, Theme(default_color=colorant"orange"))

    unshift!(layers, deutschLayer)
    unshift!(layers, bennettLayer)
    unshift!(names, "DEJMPS protocol")
    unshift!(names, "BBPSSW protocol")
    unshift!(colors, "cyan")
    unshift!(colors, "orange")


    # draw(PDF(string("plot.pdf"), 4inch, 3inch), pl);
    pl = plot(Theme(panel_fill=colorant"white"),layers...,Guide.xlabel("Probability of success"),
       Stat.xticks(ticks=collect(0:0.1:1)),
          Stat.yticks(ticks=collect(0.5:0.1:1)),
        Guide.ylabel("Fidelity"),
        Guide.title(titleName),
        Guide.manual_color_key("Legend",
        names,
        colors))
end



If you wish, save the plot 

In [ ]:
# Save the plot
filename = "Example";
draw(PNG(string(filename ,".png"), 600px, 300px), pl)
draw(PDF(string(filename ,".pdf"), 4inch, 3inch), pl)

-----
## Example Symmetric Extensions

Let us now consider some examples of deriving bounds using the PPT relaxation, enhanced by also performing symmetric extensions. EntanglementDist implements 1 and 2-extensions, which are generally feasible to compute for small instances. Note that employing extensions does not always yield improvements.

Let us for the same example compute the values from the PPT relaxation, as well as 1 and 2 extensions for comparison.

In [ ]:
# Example states

# The werner state with probability p = 0.9
#rho = wernerState(0.9);

# The EPR pair
# rho = maxEnt(2);

# The r State with probability p = 0.9
rho = rState(0.9);

# Dimensions of what is A and B for this state
nA = nB = 2;

# (Local) dimension of the maximally entangled state we want to distill
k = 2;

# Desired probability of success
delta = 0.8;

# First the PPT relaxation
(problem, Fppt, psucc) = pptRelax(rho, nA, nB, k, delta);


# 1 extension
(problem, F1ext, psucc) = pptRelax1Ext(rho, nA, nB, k, delta);


# 2 extension
(problem, F2ext, psucc) = pptRelax2Ext(rho, nA, nB, k, delta);

print("\n\nComputing relaxations for success probability p_succ=", delta, "\n");
print("PPT Relaxation: F <= ", round(Fppt,3), "\n");
print("1-Extension + PPT: F <= ", round(F1ext,3), "\n");
print("2-Extension + PPT: F <= ", round(F2ext,3),"\n");

---- 
## Example Seesaw Optimization

While the above methods are concerned with deriving upper bounds on the fidelity, the seesaw method tries to find a better protocol starting from an existing distillation protocol. 

EntanglementDist implements such a seesaw optimization. The existing protocol must use realistic operations. That is, it consists of a map $\Lambda_{A\rightarrow \hat{A}, F_A}$ for Alice and $\Lambda_{B\rightarrow \hat{B},F_B}$ for Bob, followed by a measurement of the flag registers $F_A$, $F_B$ to decide to success or failure that is implemented by a local measurement plus classical communication. For example, success may be decided by the measurement $\{P,I-P\}$, where $P = |11\rangle\langle 11|_{F_A,F_B}$. 

The seesaw function takes as inputs the Choi states of Alice and Bob 

$
C_{\hat{A},F_A,A'} = \Lambda_{A\rightarrow \hat{A},F_A} \otimes I_{A'}(\Phi_{AA'})\\
C_{\hat{B},F_B,B'} = \Lambda_{B\rightarrow \hat{B},F_B} \otimes I_{B'}(\Phi_{BB'})
$

where $\Phi_{AA'}$ and $\Phi_{BB'}$ is the normalized maximally entangled state across $A, A'$ and $B,B'$, where $A'$ and $B'$ are copies of $A$ and $B$ respectively. In addition, it demands the projector $P$ deciding success.

The following example starts from a well known filtering scheme, here adapted to the state 

$
\rho = p |\Phi\rangle\langle \Phi| + (1-p) |01\rangle\langle 01|
$

In terms of a generalized measurement, this scheme is written as Alice performing a measurement $\{M_A^{ok},I-M_A^{ok}\}$ with $M_A^{ok} = \sqrt{\epsilon} |0\rangle\langle 0| + |1\rangle \langle 1|$, and Bob a measurement given by
$\{M_B^{ok},I-M_B^{ok}\}$ with $M_B^{ok} = \sqrt{\epsilon} |1\rangle\langle 1| + |0\rangle \langle 0|$. If both obtain outcome ``ok``, they declare success. It is easy to obtain choi states for such schemes using the functionality of EntanglementDist by using measureSchemeMakeChoi, or measureScheme to actually compute its performance. For the filtering scheme, this is already implemented using the function filteringMakeChoi as used in the example below.

In [ ]:
# Let's set the parameter epsilon in the filtering protocol (See above)
eps = 0.2;

# Customized state: here the filtering is always better, but the seesaw
# even better
p = 0.7
rho = p * maxEnt(2) + (1-p) * eVec(4,1)*eVec(4,1)';

# Output initial fidelity
Finit = entFidelity(rho);
print("Initial fidelity is F=", round(Finit,3), "\n");

# Test the seesaw optimization for the filtering scheme
# Adapted to work well for states of the form p EPR + (1-p) |01><01|
print("Testing the seesaw method\n");
print("Epsilon is set to ", eps, "\n");

print("Computing Choi states\n");
(CA,CB) = filteringMakeChoi(eps);
print("Compute the filtering performance without Choi state to double check:\n");
(rhoQC, P, Fwo, pwo) = filtering(rho, eps);
print("Checked F=",round(Fwo,3), " and psucc=",round(pwo,3),"\n");

print("Run the SEESAW Optimization\n");
(newCA, newCB, Fnew, pnew) = seesaw(rho,2,2,2,CA, CB, P);